#Lab.06 / IBM3202 – Molecular Docking on AutoDock

###Theoretical aspects

As more and more protein structures are determined experimentally using X-ray crystallography or nuclear magnetic resonance (NMR) spectroscopy, molecular docking is increasingly used as a tool in **drug discovery**.

**Molecular docking simulations** explore the potential binding poses of small molecules on the **binding site** of a target protein for which an experimentally determined structure is available.
Docking against protein targets generated by **comparative modelling** also becomes possible for proteins whose structures are yet to be solved.

Thus, the **_druggability_** of different compounds and their binding affinity on a given protein target can be calculated for further lead optimization processes.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/docking_01.png'/>
<figcaption>FIGURE 1. In molecular docking, binding is evaluated in two steps: A) Energetics of the transition of the unbound states of ligand and target towards the conformations of the bound complex; and B) energetics of protein-ligand binding in these conformations. <br> Huey R et al (2007) <i>J Comput Chem 28(6), 1145-1152.</i></figcaption></center>
</figure>

Molecular docking programs perform a **search algorithm** in which varying conformations of a given ligand, typically generated using Monte Carlo or Genetic algorithms, are recursively evaluated until convergence to an energy minimum is reached. Finally, through an **affinity scoring function**, a ΔG [binding free energy in kcal/mol] is estimated and employed to rank the candidate poses as the sum of several energetic contributions (electrostatics, van der Waals, desolvation, etc).

##Experimental Overview

Typically, in this laboratory session we dock a dimeric analog of the plastic polymer PET (2PET) onto the binding site of PET hydrolase from *I. sakaiensis*, a PET-degrading enzyme discovered in Japan in 2016 for which our lab solved its three-dimensional structure [Fecker T et al (2018) *Biophys J 114 (6), 1302-1312*].

However, inspired by the COVID-19 pandemic, in this laboratory session we will perfom a docking assay of **indinavir**, an active component of the **antiretroviral therapy to treat HIV**, onto the binding site of its target protein, **HIV-2 protease**.

For our laboratory session, we will install and use **MGLtools** (and alternatively **pdb2pqr**) to prepare the target protein files, **OpenBabel** to prepare the ligand files, **AutoDock Vina** for the docking procedure and **py3Dmol** to establish the appropriate search grid configuration and analyze the results.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/docking_02.png' />
<figcaption>FIGURE 2. General steps of molecular docking. First, the target protein and ligand or ligands are parameterized. Then, the system is prepared by setting up the search grid. Once the docking calculation is performed, ligand poses are scored based on a given energy function. Lastly, the computational search is processed and compared against experimental data for validation <br><i>Taken from Pars Silico (en.parssilico.com).</i></figcaption></center>
</figure>

#Part 0 – Downloading and Installing the required software

Before we start, you must first **remember to start the hosted runtime in Google Colab**.

Then, we must install several pieces of software to perform this tutorial. Namely:
- **biopython** for manipulation of the PDB files
- **py3Dmol** for visualization of the protein structure and setting up the search grid.
- **miniconda**, a free minimal installer of **conda** for software package and environment management.
- **OpenBabel** for parameterization of our ligand(s).
- **MGLtools** for parameterization of our target protein using Gasteiger charges.
- **pdb2pqr** for parameterization of our protein using the AMBER ff99 forcefield.
- **Autodock Vina** for the docking process

After several tests, the following installation instructions are the best way of setting up **Google Colab** for this laboratory session.

1. We will first install biopython, py3Dmol, and pdb2pqr as follows:

In [49]:
# Update and reinstall libtinfo6
!sudo apt-get update
!sudo apt-get install -y libtinfo6

# Fix symbolic link
!sudo ln -sf /usr/lib/x86_64-linux-gnu/libtinfo.so.6 /usr/local/lib/libtinfo.so.6

# Add library path (if necessary)
%env LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH

# Verify installation
!ls -l /usr/local/lib/libtinfo.so.6
!echo $LD_LIBRARY_PATH


#Installing py3Dmol using pip
!pip -q install py3Dmol
#Installing biopython using pip
!pip -q install biopython
#Installing pdb2pqr v3.0 using pip
!pip -q install pdb2pqr
#We will also install RDkit
!pip -q install rdkit-pypi

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [50]:
#Importing py3Dmol for safety
import py3Dmol

In [51]:
#Checking that pdb2pqr was properly installed
!pdb2pqr30 -h | awk 'NR==1{if($1=="usage:") print "PDB2PQR succesfully installed"; else if($1!="usage:") print "Something went wrong. Please install again"}'

PDB2PQR succesfully installed


2. And then we will install conda to be able to install MGLtools and OpenBabel

In [73]:
#Install conda using the new conda-colab library
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

#Install MGLtools and OpenBabel from
#the bioconda repository
!conda install -c conda-forge -c bioconda mgltools openbabel zlib ncurses --yes

#!sudo apt-get update
#!sudo apt-get install -y openbabel
#!obabel -V


✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mgltools
    - ncurses
    - openbabel
    - zlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    archspec-0.2.3             |     pyhd8ed1ab_0          48 KB  conda-forge
    ca-certificates-2024.12.14 |       hbcca054_0         153 KB  conda-forge
    cairo-1.18.2               |       h3394656_1         956 KB  conda-forge
    certifi-2024.12.14         |     pyhd8ed1ab_0         158 KB  conda-forge
    conda-24.11.3              |  py310hff52083_0         906 KB  conda-forge
    font-ttf-dejavu-sans-mono-2

3. Finally, we will download the Autodock Vina program from the [AutoDock Vina GitHub](https://github.com/ccsb-scripps/AutoDock-Vina) and make an alias to use it during this session

In [74]:
#Download and extract Autodock Vina from SCRIPPS
#Then, we set up an alias for vina to be treated as a native binary

%%bash
# Remove existing 'vina' directory if it exists
if [ -d "vina" ]; then
  rm -rf vina
fi

%%bash
mkdir vina
cd vina
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
chmod +x /content/vina/vina_1.2.5_linux_x86_64
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_split_1.2.5_linux_x86_64
chmod +x /content/vina/vina_split_1.2.5_linux_x86_64

bash: line 6: fg: no job control


In [75]:
%alias vina /content/vina/vina_1.2.5_linux_x86_64
%alias vina_split /content/vina/vina_1.2.5_linux_x86_64

Once these software installation processes are completed, we are ready to perform our experiments

#Part 1 – Downloading and Preparing the Receptor for AutoDock

1. The first step in a molecular docking procedure is to have a structure of a given target protein. While in some cases a high-quality comparative model can be used, most cases start with an experimentally (X-ray, NMR, cryoEM) solved three-dimensional structure.

  In such scenario, a given target protein structure is downloaded from the **Protein Data Bank (PDB)** (https://www.rcsb.org/pdb) using a given accession ID. For example, the PET hydrolase solved by our lab has the accession ID 6ANE.

  For this tutorial, we will use the structure of the HIV-2 protease, solved using X-ray crystallography and deposited in the PDB with the accession ID 1HSG, which we will download using **biopython**:

In [76]:
#Downloading the PDB files using biopython
import os
from Bio.PDB import *
pdbid = ['1hsg']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

2. In the case of X-ray diffraction, this experimental strategy does not discriminate between electron density coming from static protein atoms or water molecules, meaning that most protein structures solved by X-ray diffraction also include so-called **crystallographic waters** (check the non-bonded red dots on the protein structure below). These molecules are not important for our particular docking simulation, and we also have to remove them.
<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/docking_03.png' />
<figcaption>FIGURE 3. Cartoon representation of HIV-2 protease dimer (PDB accession ID 1HSG), with its N-to-C-terminal residues colored from blue to red in rainbow spectrum, showing the crystallographic waters as red spheres.</i></figcaption></center>
</figure>

  Typically, this can be easily done by extracting all of the lines from the PDB file that start with **"ATOM"**, as this is how all of the atoms that belong to amino and nucleic acid residues are termed. In contrast, the atoms from ligands, water molecules and other non-protein/non-nucleic residues are commonly referred to as **"HETATM"**. Also, the different chains of an oligomer are separated by a **"TER"** string, which is important to keep in our case.

  The following **python script** will first create a folder in which we will store all data related to our molecular docking experiment. Then, it will extract all lines matching the string "ATOM" (for the protein atoms) or "TER" (for the chain separations) into a separate PDB file for further processing. Please take a good look at it.

In [77]:
#This script will create a folder called "single-docking" for our experiment
#Then, it will print all "ATOM" and "TER" lines from a given PDB into a new file

#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the working directory
singlepath = Path("/content/drive/MyDrive/AlphaFold2_Results")

# Input and output file names
protein = "R9_ATR75_unrelaxed_rank_001.pdb"
input_file = singlepath / protein
output_file = singlepath / "R9_ATR75_unrelaxed_rank_001_prot.pdb"

# Check if the input file exists
if not input_file.exists():
    raise FileNotFoundError(f"Input file not found: {input_file}")

# Process the file
# Considered PDB file uses Fixed Width Format
try:
    with open(input_file, "r") as f, open(output_file, "w") as g:
        for line in f:
            # Use slicing to check for "ATOM" or "TER" in the first 4 characters
            if line[:4].strip() == "ATOM":
                g.write(line)
            elif line[:4].strip() == "TER":
                g.write(line)
        g.write("END\n")  # Add END at the end
    print(f"File successfully created: {output_file}")
except Exception as e:
    print(f"An error occurred: {e}")

Mounted at /content/drive
File successfully created: /content/drive/MyDrive/AlphaFold2_Results/R9_ATR75_unrelaxed_rank_001_prot.pdb


3. Once we printed out the "ATOM" lines of the parent PDB file, we have a new file that contains  the coordinates of our target protein.

  However, for AutoDock Vina to perform a molecular docking experiment, the target protein file must contain atom types compatible with Autodock for evaluating different types of interaction, as well as their partial charges to evaluate electrostatic interactions. Such information is included in a file known as **PDBQT**, a modification of the PDB format that also includes **charges (q)** and **AutoDock-specific atom types (t)** in two extra columns at the end of the now PDBQT file. It is worth noting, however, that Autodock Vina **ignores the user-supplied partial charges**, as it has its own way of dealing with electrostatic interactions.

  Lastly, the protein target must contain **all polar hydrogens**, such that hydrogen bonds can be formed between the target protein and the ligand. Most protein structures have no hydrogens included, meaning that we must add them.

  This is the part of the tutorial where you have **two different options** to proceed with your experiment!

3. a) Add the polar hydrogens of your protein and parameterize it with **Gasteiger** charges and atom types using **MGLtools** (this is the canonical option for the majority of AutoDock users)

In [67]:
!sudo apt-get update
!sudo apt-get install -y python2

# Create a symbolic link to /usr/local/bin/
!sudo ln -s /usr/bin/python2 /usr/local/bin/python2

# Verify installation
!ls /usr/local/bin/python*
!python2 --version


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [78]:
#Parameterizing and adding Gasteiger charges into our protein using MGLtools
#!python2 /usr/local/bin/prepare_receptor4.py -r $singlepath/1hsg_prot.pdb -o $singlepath/1hsg_prot.pdbqt -A hydrogens -U nphs_lps -v
outputpath="/content/drive/MyDrive/AutoDock_Vina_Results"

!python2 /usr/local/bin/prepare_receptor4.py \
  -r "$singlepath/R9_ATR75_unrelaxed_rank_001_prot.pdb" \
  -o "$outputpath/R9_ATR75_unrelaxed_rank_001_prot.pdbqt" \
  -A hydrogens -U nphs_lps -v

set verbose to  True
read  /content/drive/MyDrive/AlphaFold2_Results/R9_ATR75_unrelaxed_rank_001_prot.pdb
setting up RPO with mode= automatic and outputfilename=  /content/drive/MyDrive/AutoDock_Vina_Results/R9_ATR75_unrelaxed_rank_001_prot.pdbqt
charges_to_add= gasteiger
delete_single_nonstd_residues= None
adding gasteiger charges to peptide


3. b) Add the polar hydrogens of your protein and parameterize it based on the pKa of each aminoacid at pH 7.4 with the **AMBER99ff** force field using **pdb2pqr**, followed by deletion of non-polar hydrogens and conversion into **PDBQT** file using **MGLtools**.

  In this case, pdb2pqr generates an intermediate **PQR** file, a modification of the PDB format which allows users to add charge and radius parameters to existing PDB data. This information is then unaltered during the use of **MGLtools**.

In [79]:
#First, using pdb2pqr to parameterize our receptor with AMBER99ff, maintaining
#the chain IDs and setting up the receptor at a pH of 7.4
#!pdb2pqr30 --ff AMBER --keep-chain --titration-state-method propka --with-ph 7.4 $singlepath/1hsg_prot.pdb $singlepath/1hsg_prot.pqr

!pdb2pqr30 --ff AMBER --keep-chain --titration-state-method propka --with-ph 7.4 \
$singlepath/R9_ATR75_unrelaxed_rank_001_prot.pdb $outputpath/R9_ATR75_unrelaxed_rank_001_prot.pqr

# Then, convert the .pqr file into a .pdbqt file
!python2 /usr/local/bin/prepare_receptor4.py -r $outputpath/R9_ATR75_unrelaxed_rank_001_prot.pqr \
-o $outputpath/R9_ATR75_unrelaxed_rank_001_prot.pdbqt -C -U nphs_lps -v


INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: /content/drive/MyDrive/AlphaFold2_Results/R9_ATR75_unrelaxed_rank_001_prot.pdb
INFO:Setting up molecule.
INFO:Created biomolecule object with 305 residues and 2336 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 1 missing atoms in biomolecule.
INFO:Added atom OXT to residue TRP A 305 at coordinates 18.857, -11.466, -6.685
INFO:Updating disulfide bridges.
INFO:Debumping biomolecule.
INFO:Assigning titratio

**You are all set with your target protein!**

>Before we move onto preparing the ligand for molecular docking, please consider the following questions:
- Why is it important to add hydrogens for the purposes of our docking simulations?
- Why are we only adding polar hydrogens?
- If the partial charges are ignored by Autodock Vina, how can these different strategies affect my docking results? (this is something you can actually test!)



#Part 2 – Downloading and Preparing the Ligand for AutoDock

1. We now need to prepare the ligand that we will use for our docking analysis. In our case, we will use **Indinavir**. This drug is a protease inhibitor used as a component of the antiretroviral therapy to treat HIV/AIDS, aiding in decreasing the viral load. In this opportunity, we will attempt to predict the docking pose of indinavir onto the binding site of the HIV-2 protease.

  We will first start by creating a folder in which we will store our ligands for molecular docking.

In [80]:
#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#We will first create a path for all ligands that we will use in this tutorial
#Notice that the HOME folder for a hosted runtime in colab is /content/
ligandpath = Path("/content/drive/MyDrive/Ligands/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(ligandpath):
    print("Ligand path already exists")
else:
    os.mkdir(ligandpath)
    print("Ligand path was successfully created")

Ligand path already exists


2. Now, we will download indinavir from the **DrugBank** database (*Nucleic Acids Res
. 2006; 34, D668-72*). This is comprehensive, freely accessible, online database containing information on drugs and drug targets. You can actually check the detailed chemical, pharmacological and pharmaceutical information on Indinavir [in this DrugBank link](https://www.drugbank.ca/drugs/DB00224).

  We will download this ligand in SMILES format to continue with its preparation for molecular docking



3. Hey! But what is a SMILES format? Well, the **Simplified Molecular-Input Line-Entry System** (SMILES) is a text notation that allows a user to represent a chemical structure in a way that can be used by the computer. The elemental notation for different types of bonds between different atoms is as follows:

  \-	for single bonds (eg. C-C or CC is CH3CH3)

  \=	for double bonds (eg. C=C for CH2CH2)

  \#	for triple bonds (eg. C#N for C≡N)

  \	for aromatic bond (eg. C\*1\*C\*C\*C\*C\*C1 or c1ccccc1 for benzene)

  \. for disconnected structures (eg. Na.Cl for NaCl)

  / and \ for double bond stereoisomers (eg. F/C=C/F for trans-1,2-difluoroethylene and F/C=C\F for cis-1,2-difluoroethylene)

  @ and @@ for enantiomers (eg. N\[C@@H](C)C(=O)O for L-alanine and N\[C@H](C)C(=O)O for D-alanine)

  **Let's take a look at the SMILES of Indinavir!**

In [83]:
#@title Use the following Viewer to load your SMILES as a 3D molecule
import py3Dmol
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

smiles_data = "CCCCCCCCCCCCCCCC(=O)SCCNC(=O)CCNC(=O)[C@@H](C(C)(C)COP(=O)([O-])OP(=O)([O-])OC[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)([O-])[O-])O"


def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view()
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

from ipywidgets import interact,fixed,IntSlider
import ipywidgets


def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None

@interact
def smi2viewer(smiles=fixed(smiles_data)):  # `smiles_data`를 직접 사용
    try:
        print(f"SMILES Data: {smiles}")  # SMILES 데이터 출력
        conf = smi2conf(smiles)
        return MolTo3DView(conf).show()
    except Exception as e:
        print(f"Failed to generate 3D structure: {e}")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

4. Now, we will take this SMILES format and use it to construct and parameterize a three-dimensional structure of Indinavir in **PDBQT** format for its use in molecular docking. As with the receptor, we also have different options to prepare our ligand for molecular docking:

4. a) Use the program **babel** to convert the SMILES into a **MOL2** file without any extra work (such as searching for best conformers) except for setting the protonation state to pH 7.4, and then use **MGLtools** to parameterize the ligand using **Gasteiger** partial charges (this is the canonical option for the majority of AutoDock users). Please note that we are generating a ligand in which **all torsions are active** during the docking procedure.

In [93]:
import os
import subprocess
from pathlib import Path

#Converting Indinavir from SMILES into a 3D PDB format
#NOTE: for some reason, MGLtools does not recognize the ligand when inside a different folder
#Here we are deleting the temporary PDB file required for generating the PDBQT fileS

smiles_data = "CCCCCCCCCCCCCCCC(=O)SCCNC(=O)CCNC(=O)[C@@H](C(C)(C)COP(=O)([O-])OP(=O)([O-])OC[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)([O-])[O-])O"
mol2_file = singlepath / "Hexadecanoyl_CoA.mol2"

try:
    result = subprocess.run(
        ["obabel", "-:", smiles_data, "-O", str(mol2_file), "--gen3d", "-v"],
        text=True,
        capture_output=True,
        check=True
    )
    print(f"Open Babel Output:\n{result.stdout}")

    if mol2_file.exists():
        print(f"MOL2 file successfully created: {mol2_file}")
    else:
        print(f"Failed to create MOL2 file: {mol2_file}")
except subprocess.CalledProcessError as e:
    print(f"Error while running Open Babel:\n{e.stderr}")


Open Babel Output:

MOL2 file successfully created: /content/drive/MyDrive/AlphaFold2_Results/Hexadecanoyl_CoA.mol2


In [109]:
print(f"MOL2 File Path: {mol2_file}")
print(f"File Exists: {os.path.exists(mol2_file)}")

!pip install mgltools
import os
import subprocess

mol2_file = os.path.abspath(singlepath / "Hexadecanoyl_CoA.mol2")
pdbqt_file = os.path.abspath(singlepath / "Hexadecanoyl_CoA.pdbqt")

print(f"Absolute MOL2 File Path: {mol2_file}")
print(f"File Exists: {os.path.exists(mol2_file)}")

try:
    # File reading test
    with open(mol2_file, "r") as file:
        print("MOL2 File Content:")
        print(file.read())

    # Script execution
    result = subprocess.run(
        [
            "python2", "/usr/local/bin/prepare_ligand4.py",
            "-l", mol2_file,
            "-o", pdbqt_file,
            "-U", "nphs_lps", "-v"
        ],
        text=True,
        capture_output=True,
        check=True
    )
    print(f"MGLTools Output:\n{result.stdout}")

    if os.path.exists(pdbqt_file):
        print(f"PDBQT file successfully created: {pdbqt_file}")
    else:
        print(f"Failed to create PDBQT file: {pdbqt_file}")
except Exception as e:
    print(f"Error: {e}")

import os

print(f"Input file path: {mol2_file}")
print(f"Output file path: {pdbqt_file}")
print(f"File exists (MOL2): {os.path.exists(mol2_file)}")
print(f"File exists (PDBQT): {os.path.exists(pdbqt_file)}")


MOL2 File Path: /content/drive/MyDrive/AlphaFold2_Results/Hexadecanoyl_CoA.mol2
File Exists: True
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: Could not find a version that satisfies the requirement mgltools (from versions: none)
ERROR: No matching distribution found for mgltools
Absolute MOL2 File Path: /content/drive/MyDrive/AlphaFold2_Results/Hexadecanoyl_CoA.mol2
File Exists: True
MOL2 File Content:

Error: Command '['python2', '/usr/local/bin/prepare_ligand4.py', '-l', '/content/drive/MyDrive/AlphaFold2_Results/Hexadecanoyl_CoA.mol2', '-o', '/content/drive/MyDrive/AlphaFold2_Results/Hexadecanoyl_CoA.pdbqt', '-U', 'nphs_lps', '-v']' returned non-zero exit status 1.
Input file path: /content/drive/MyDrive/AlphaFold2_Results/Hexadecanoyl_CoA.mol2
Output file path: /content/drive/MyDrive/AlphaFold2_Results/Hexadecanoyl_CoA.pdbqt
File exists

In [99]:
#Converting Indinavir from SMILES into a 3D MOL2 format and perform a weighted rotor search for lowest energy conformer
#Then, prepare ligand for docking using the Autodock script

pdbqt_file = singlepath / "Hexadecanoyl_CoA.pdbqt"
!chmod +x /usr/local/bin/prepare_ligand4.py

# Step 2: MGLTools를 사용해 MOL2 파일을 PDBQT 파일로 변환
try:
    result = subprocess.run(
        [
            "python2", "/usr/local/bin/prepare_ligand4.py",
            "-l", str(mol2_file),
            "-o", str(pdbqt_file),
            "-U", "nphs_lps", "-v"
        ],
        text=True,
        capture_output=True,
        check=True
    )
    print(f"MGLTools Output:\n{result.stdout}")

    if pdbqt_file.exists():
        print(f"PDBQT file successfully created: {pdbqt_file}")
    else:
        print(f"Failed to create PDBQT file: {pdbqt_file}")
except FileNotFoundError:
    print("Python2 is not installed or prepare_ligand4.py script is missing.")
except subprocess.CalledProcessError as e:
    print(f"Error while running MGLTools:\n{e.stderr}")

# Step 3: 필요 없는 MOL2 파일 삭제
#try:
#    if mol2_file.exists():
#        os.remove(mol2_file)
#        print(f"Temporary MOL2 file deleted: {mol2_file}")
#except Exception as e:
#    print(f"Error while deleting MOL2 file: {e}")

Error while running MGLTools:
Traceback (most recent call last):
  File "/usr/local/bin/prepare_ligand4.py", line 185, in <module>
    mols = Read(ligand_filename)
  File "/usr/local/MGLToolsPckgs/MolKit/__init__.py", line 21, in Read
    raise AssertionError , "%s does't exist" %filename
AssertionError: Hexadecanoyl_CoA.mol2 does't exist



4. b) Use the program **babel** to  convert the SMILES into a 3D **MOL2** file while simultaneously performing and energy minimization using the Generalized Amber Force Field (**GAFF**). Then, use **MGLtools** to parameterize the ligand using **Gasteiger** partial charges. Please note that we are generating a ligand in which **all torsions are active** during the docking procedure.

In [ ]:
#Converting Indinavir from SMILES into a 3D MOL2 format and perform an energy minimization of the conformer using the GAFF forcefield
#Then, prepare ligand for docking using the Autodock script
#!obabel $ligandpath/DB00224.smiles -O indinavir.mol2 --gen3d --best --canonical --minimize --ff GAFF --steps 10000 --sd
#!obabel /content/ligands/DB02338.smiles.txt -O nadph.mol2 --gen3d --best --canonical --minimize --ff GAFF --steps 10000 --sd
!obabel $ligandpath/Hexadecanoyl_CoA.smiles.txt -O $ligandpath/Hexadecanoyl_CoA.mol2 \
  --gen3d --best --canonical --minimize --ff GAFF --steps 1000 --sd

#!python2 /usr/local/bin/prepare_ligand4.py -l indinavir.mol2 -o $singlepath/indinavir.pdbqt -U nphs_lps -v
!python2 /usr/local/bin/prepare_ligand4.py \
  -l $ligandpath/Hexadecanoyl_CoA.mol2 \
  -o $outputpath/Hexadecanoyl_CoA.pdbqt \
  -U nphs_lps -v

#os.remove("indinavir.mol2")
mol2_file = f"{ligandpath}/Hexadecanoyl_CoA.mol2"
if os.path.exists(mol2_file):
    os.remove(mol2_file)
    print(f"Temporary file removed: {mol2_file}")
else:
    print(f"No temporary file to remove: {mol2_file}")

4. c) Use the program **babel** to  convert the SMILES into a 3D **MOL2** file while simultaneously performing a weighted rotor search for the lowest energy conformer using the Generalized Amber Force Field (**GAFF**). Then, use **MGLtools** to parameterize the ligand using **Gasteiger** partial charges. Please note that we are generating a ligand in which **all torsions are active** during the docking procedure.

In [ ]:
#Converting Indinavir from SMILES into a 3D MOL2 format and perform a weighted rotor search for lowest energy conformer
#Then, prepare ligand for docking using the Autodock script
#!obabel $ligandpath/DB00224.smiles -O indinavir.mol2 --gen3d --best --canonical --conformers --weighted --nconf 50 --ff GAFF
#!python2 /usr/local/bin/prepare_ligand4.py -l indinavir.mol2 -o $singlepath/indinavir.pdbqt -U nphs_lps -v
#os.remove("indinavir.mol2")

!obabel -:"NC(=O)C1=CN(C=CC1)[C@@H]1O[C@H](COP(O)(=O)OP(O)(=O)OC[C@H]2O[C@H]([C@H](OP(O)(O)=O)[C@@H]2O)N2C=NC3=C2N=CN=C3N)[C@@H](O)[C@H]1O" \
-O nadph.mol2 --gen3d --best --canonical --conformers --weighted --nconf 50 --ff GAFF
!python2 /usr/local/bin/prepare_ligand4.py -l nadph.mol2 -o $singlepath/nadph.pdbqt -U nphs_lps -v
os.remove("nadph.mol2")


**You are all set with your ligand!** Now, we move onto setting up the molecular docking experiment

#Part 3 – Setting up and Performing Molecular Docking with AutoDock

1. As explained in the lectures, it is necessary to define the search space for molecular docking on a given target protein through the use of a **grid box**. This grid box is usually centered within the binding, active or allosteric site of the target protein and its size will be sufficiently large such that **all binding residues are placed inside the grid box**.

  Here, we will make use of **py3Dmol** to visually inspect the protein structure in cartoon representation and to draw a grid box. The position and size of the grid box will be defined by the coordinates of its centroid and by its dimensions in x, y and z.

  To better guide the search for the optimal dimensions and coordinates of the grid box, we will also show the residues Val32, Ile47 and Val82 of HIV-2 protease.

  The script that defines the visualizer, which we called **ViewProtGrid**, is first loaded into **Colab** with the following lines of code

In [ ]:
#@title Loading the script that creates our Docking Box Viewer
#These definitions will enable loading our protein and then
#drawing a box with a given size and centroid on the cartesian space
#This box will enable us to set up the system coordinates for the simulation
#
#HINT: The active site of the HIV-2 protease is near the beta strands in green
#
#ACKNOWLEDGE: This script is largely based on the one created by Jose Manuel
#Napoles Duarte, Physics Teacher at the Chemical Sciences Faculty of the
#Autonomous University of Chihuahua (https://github.com/napoles-uach)
#
#First, we define the grid box
def definegrid(object,bxi,byi,bzi,bxf,byf,bzf):
  object.addBox({'center':{'x':bxi,'y':byi,'z':bzi},'dimensions': {'w':bxf,'h':byf,'d':bzf},'color':'blue','opacity': 0.6})

#Next, we define how the protein will be shown in py3Dmol
#Note that we are also adding a style representation for active site residues
def viewprot(object,prot_PDBfile,resids):
  prot_PDBfile = "/content/R1_seq2_unrelaxed_rank_001.pdb" # 추가
  mol1 = open(prot_PDBfile, 'r').read()
  object.addModel(mol1,'pdb')
  object.setStyle({'cartoon': {'color':'spectrum'}})
  resids = list(range(26, 81))  # R9 seq1 block2 residue
  object.addStyle({'resi':resids},{'stick':{'colorscheme':'greenCarbon'}})

#Lastly, we combine the box grid and protein into a single viewer
def viewprotgrid(prot_PDBfile,resids,bxi,byi,bzi,bxf=10,byf=10,bzf=10):
  mol_view = py3Dmol.view(1000,1000,viewergrid=(1,2))
  definegrid(mol_view,bxi,byi,bzi,bxf,byf,bzf)
  viewprot(mol_view,prot_PDBfile,resids)
  mol_view.setBackgroundColor('0xffffff')
  mol_view.rotate(90, {'x':0,'y':1,'z':0},viewer=(0,1));
  mol_view.zoomTo()
  mol_view.show()


2. Now, we will use our ViewProtGrid to visualize the protein, binding site residues and a grid box of variable size and position that we can manipulate using a slider through *ipywidgets*. You have to enter the location of the PDB file in the *pdbfile* variable (e.g. single-dock/1hsg_prot.pdb) and the residues that you want to show from the PDB in the *resids* variable.


Examples of how to use the *pdbfile* variable
>pdbfile = 1hsg_prot.pdb (if the PDB file is in the current path)

>pdbfile = single-dock/1hsg_prot.pdb (if the PDB file is in the single-dock folder)

Examples of how to use the *resids* variable

>resids = [82] shows a single residue in position 82)

>resids = [82,83,84] shows residues 82, 83 or 84 separately, which you can select in the viewer

>resids = [(82,83,84)] shows residue 82, 83 and 84 in the same visualization

>resids = ['82-84'] shows residue range 82-84 in the same visualization

**NOTE:** This code fails when attempting to show two non-consecutive residues in the same visualization.


In [ ]:
#@title Loading the Docking Box viewer for a user-defined PDB input file and highlighted residues
from ipywidgets import interact,fixed,IntSlider
import ipywidgets

pdbfile = "/content/R1_seq2_unrelaxed_rank_001.pdb" # @param {type:"string"}
resids = ['26-80'] # @param {type:"raw"}
interact(viewprotgrid,
#ADD YOUR PDB LOCATION AND FILENAME HERE
         prot_PDBfile = pdbfile,
#ADD THE RESIDUES YOU WANT TO VISUALIZE HERE
         resids = resids,
         bxi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         byi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bzi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bxf=ipywidgets.IntSlider(min=4,max=40, step=1),
         byf=ipywidgets.IntSlider(min=4,max=30, step=1),
         bzf=ipywidgets.IntSlider(min=4,max=30, step=1))

3. Now, we will generate a configuration file for **Autodock Vina**. As expected, the configuration file contains information about the target protein and ligand, as well as the position and dimensions of the grid box that defines the search space.

  For defining the grid box, you will use the  box origin and size coordinates that you defined manually in the previous step.

  The following is an example file of a standard **Autodock Vina configuration file**, including several possible variables that can be edited. A very relevant variable is the **exhaustiveness**, i.e., the number of independent runs starting from random conformations, and therefore the amount of computational effort during molecular docking.


```
#CONFIGURATION FILE

#INPUT OPTIONS
receptor = [target protein pdbqt file]
ligand = [ligand pdbqt file]
flex = [flexible residues in receptor in pdbqt format]

#SEARCH SPACE CONFIGURATIONS
#Center of the box (coordinates x, y and z
center_x = [value]
center_y = [value]
center_z = [value]
#Size of the box (dimensions in x, y and z)
size_x = [value]
size_y = [value]
size_z = [value]

#OUTPUT OPTIONS
#out = [output pdbqt file for all conformations]
#log = [output log file for binding energies]

#OTHER OPTIONS
cpu = [value] # more cpus reduces the computation time
exhaustiveness = [value] # search time for finding the global minimum, default is 8
num_modes = [value] # maximum number of binding modes to generate, default is 9
energy_range = [value] # maximum energy difference between the best binding mode and the worst one displayed (kcal/mol), default is 3
seed = [value] # explicit random seed, not required
```

The following script will create this file for our docking procedure. **You will need to add the position and dimensions of your grid box**


In [ ]:
#@title Generating an AutoDock Vina file
receptor = "/content/single-dock/R1_seq2_unrelaxed_rank_001_prot.pdbqt" # @param {type:"string"}
ligand = "/content/single-dock/nadph.pdbqt" # @param {type:"string"}
center_x = "2.638" # @param {type:"string"}
center_y = "5.139" # @param {type:"string"}
center_z = "12.286" # @param {type:"string"}
size_x = "23" # @param {type:"string"}
size_y = "23" # @param {type:"string"}
size_z = "23" # @param {type:"string"}
with open(singlepath / "config_singledock","w") as f:
  f.write("#CONFIGURATION FILE (options not used are commented) \n")
  f.write("\n")
  f.write("#INPUT OPTIONS \n")
  f.write("receptor = " + receptor +"\n")
  f.write("ligand = " + ligand +"\n")
  f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
  f.write("#SEARCH SPACE CONFIGURATIONS \n")
  f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES
  f.write("center_x = " + center_x + "\n")
  f.write("center_y = " + center_y + "\n")
  f.write("center_z = " + center_z + "\n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
  f.write("#Size of the box (values bxf, byf and bzf) \n")
  f.write("size_x = " + size_x + "\n")
  f.write("size_y = " + size_y + "\n")
  f.write("size_z = " + size_z + "\n")
  f.write("#OUTPUT OPTIONS \n")
  f.write("#out = \n")
  f.write("#log = \n")
  f.write("\n")
  f.write("#OTHER OPTIONS \n")
  f.write("#cpu =  \n")
  f.write("#exhaustiveness = \n")
  f.write("#num_modes = \n")
  f.write("#energy_range = \n")
  f.write("#seed = ")

4. Lastly, we will enter into the folder that we created for the docking experiment and **perform our first molecular docking with Autodock Vina**.

  Once you execute the lines of code shown below, Autodock will show you a progress bar (if running as expected). **This simulation should not take longer than 5 min**.
  
  Note that we are defining the filenames of the output and log file outside the configuration file.

In [ ]:
#Changing directory to the single docking folder
os.chdir(singlepath)
#Executing AutoDock Vina with our configuration file
%vina --config config_singledock --out output.pdbqt | tee output.log
#Exiting the execution directory
os.chdir("/content/")

5. Once the molecular docking has finished running, we will compare the docking poses with the experimentally solved pose for indinavir. In fact, the structure of HIV-2 protease that you downloaded at the beginning of this tutorial was solved with indinavir bound to it.

  The following lines of code are similar to what we did with extracting the 'ATOM' lines of the PDB file, but now we are extracting the lines containing **'MK1'**, the name of the ligand in this PDB file.

In [ ]:
#Here, we will be extracting Indinavir, which is present in the structure of
#HIV-2 protease (yes! this is a simulation with experimental validation!)
#The approach is similar to printing the ATOM and TER lines, but we are using
#the residue name given to the ligand in the experimentally solved structure: MK1
#protein = "1hsg.pdb"
protein = "/content/Alphafold/R1_seq1_unrelaxed_rank_001.pdb"

#with open(singlepath/"xtal_ligand.pdb","w") as g:
#  f = open(protein,'r')
#  for line in f:
#    row = line.split()
#    if "MK1" in row:
#      g.write(line)
#  g.write("END")

with open(singlepath + "/nadph_ligand.pdb", "w") as g:
    with open(protein, "r") as f:
        for line in f:
            row = line.split()
            if "NAD" in row:  # NADPH의 3-letter 코드가 "NAD"인 경우
                g.write(line)
        g.write("END")


6. We also need the different docking poses generated as a result of the molecular docking simulation. We will split these poses into separate PDB files using **babel**, starting with file numbered as 1 corresponding to the lowest-energy pose.

In [ ]:
#We need to convert our Autodock Vina results from pdbqt into pdb
#For this, we use babel
!obabel -ipdbqt $singlepath/output.pdbqt -opdb -O $singlepath/indinavir_dock.pdb -m

7. Finally, we will load our target protein into **py3Dmol**, along with any docking pose of our choice and the experimentally solved binding pose of indinavir for comparison.

In [ ]:
view=py3Dmol.view()
view.setBackgroundColor('white')
#Loading the target protein as PDB file
view.addModel(open(singlepath/'1hsg_prot.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'spectrum'}})
view.zoomTo()
#Loading the docking pose
view.addModel(open(singlepath/'indinavir_dock1.pdb', 'r').read(),'pdb')
view.setStyle({'model':1},{'stick':{'colorscheme':'greenCarbon'}})
#Loading the experimentally solved binding mode
view.addModel(open(singlepath/'xtal_ligand.pdb', 'r').read(),'pdb')
view.setStyle({'resn':'MK1'},{'stick':{}})
view.show()

How similar is your docking pose when compared to the experimentally solved one?

**📚HOMEWORK:** Remember that redocking, i.e. a molecular docking simulation in which the ligand bound to the target is used as the starting conformation for the docking procedure, is commonly used as a control. Based on this information, plan a control simulation using the ligand that you just extracted in this tutorial

>```
>#Example of preparation of the experimentally solved ligand pose for redocking
>
>os.chdir(singlepath)
>!pythonsh /usr/local/bin/prepare_ligand4.py -l xtal_ligand.pdb -o $singlepath/xtal_ind.pdbqt -A hydrogens -U nphs_lps -v
>
>#Then, you can essentially use the same configuration files.
>```



**And this is the end of the sixth tutorial!** If you want to download your results, you can compress them into a zip file for manual download.

In [ ]:
!zip -r singledocking.zip $singlepath
#By default, automatic download is enabled through the following lines
#but you need to disable your adblocker in order for it to work
from google.colab import files
files.download("/content/singledocking.zip")